In [ ]:
import pandas as pd

account_booking_test = pd.read_csv("account_booking_test.csv")
account_booking_train = pd.read_csv("account_booking_train.csv")
external_parties_test = pd.read_csv("external_parties_test.csv")
external_parties_train = pd.read_csv("external_parties_train.csv")

Delete 2 legs transactions

In [ ]:
df = pd.DataFrame(account_booking_test).drop_duplicates(subset=["transaction_reference_id"], keep=False)
print(f"reduction {1-len(df)/len(account_booking_test)}")